# Lesson 12.2 线性回归建模实验     

## 一、深度学习建模流程

![32](https://i.loli.net/2021/02/05/SlouvnpBxmJYZ4c.jpg)

## 二、线性回归的手动实现


In [9]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch.utils.tensorboard import SummaryWriter

# 自定义模块

from torchLearning import *

### 1.生成数据集

&emsp;&emsp;利用此前的数据集生成函数，创建一个真实关系为$y = 2x_1-x_2+1$，且扰动项不是很大的回归类数据集。

- y.numel() 返回y中元素的个数

In [10]:
tensorGenReg?

Signature: tensorGenReg(num_examples=1000, w=[2, -1, 1], bias=True, delta=0.01, deg=1)
Docstring:
回归类数据集创建函数。
:param num_examples: 创建数据集的数据量
:param w: 包括截距的（如果存在）特征系数向量
:param bias：是否需要截距
:param delta：扰动项取值
:param deg：方程次数
:return: 生成的特征张和标签张量
File:      g:\python学习资料2022年\codetest\pytorch&深度学习\note_pytorch\note_pytorch\torchlearning.py
Type:      function


In [11]:
torch.manual_seed(420)   

features, labels = tensorGenReg()

### 2.建模流程



- Stage 1.模型选择  
  围绕建模目标，我们可以构建一个只包含一层的神经网络进行建模。

  <img src="https://i.loli.net/2021/02/05/UXeS3wZC7xiPN14.jpg" alt="33" style="zoom:40%;" />

In [12]:
def linreg(X,w):
    return torch.mm(X,w)

- Stage 2.确定目标函数  
  我们使用MSE作为损失函数，也就是目标函数

In [13]:
def squared_loss(y_hat,y):
    num_ = y.numel()
    sse = torch.sum((y_hat.reshape(-1, 1) - y.reshape(-1, 1)) ** 2)
    return sse / num_

- Stage 3.定义优化算法  
采用小批量梯度下降进行求解，每一次迭代过程都是(参数-学习率*梯度)。

In [14]:
def sgd(params,lr):
    params.data -= lr *params.grad

---


**可微张量的in-place operation（对原对像修改操作）的相关讨论**

(1).正常情况下，可微张量的in-place operation会导致系统无法区分叶节点和其他节点的问题

In [15]:
w = torch.tensor(2., requires_grad = True)
w

tensor(2., requires_grad=True)

In [16]:
w.is_leaf

# 开启可微之后，w的所有计算都会被纳入计算图中

True

In [17]:
w1 = w * 2
w1

tensor(4., grad_fn=<MulBackward0>)

但如果在计算过程中，我们使用in-place operation，让新生成的值替换w原始值，则会报错

In [18]:
w = torch.tensor(2., requires_grad = True)
w -= w * 2

RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

PyTorch中不允许叶节点使用in-place operation，根本原因是会造成叶节点和其他节点类型混乱。不过，虽然可微张量不允许in-place operation，但却可以通过其他方法进行对w进行修改。

In [20]:
w = torch.tensor(2., requires_grad = True)
w = w * 2

# 不过此时，w就不再是叶节点了

In [21]:
w.is_leaf

False

In [22]:
# 我们也无法通过反向传播求其导数
w = torch.tensor(2., requires_grad = True)
w = w * 2
w.backward()       # w已经成为输出节点
w.grad

c:\Users\HP\Python\Python310\lib\site-packages\torch\_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten\src\ATen/core/TensorBody.h:482.)
  return self._grad


(2).叶节点数值修改方法

&emsp;&emsp;当然，如果出现了一定要修改叶节点的取值的情况，典型的如梯度下降过程中利用梯度值修改参数值时，可以使用此前介绍的暂停追踪的方法，如使用`with torch.no_grad()`语句或者`torch.detach()`方法，使得修改叶节点数值时暂停追踪，然后再生成新的叶节点带入计算，如：

In [23]:
#————————————————————————————with torch.no_grad()————————————————————————————
w = torch.tensor(2., requires_grad = True)

# 利用with torch.no_grad()暂停追踪
with torch.no_grad():
    w -= w * 2
w

tensor(-2., requires_grad=True)

In [24]:
w.is_leaf

True

In [25]:
#————————————————————————————利用detach生成新变量————————————————————————————

w = torch.tensor(2., requires_grad = True)

# 利用detach生成新变量
w.detach_()

tensor(2.)

In [26]:
w -= w * 2
w

tensor(-2.)

In [27]:
w.requires_grad = True

In [28]:
w

tensor(-2., requires_grad=True)

In [29]:
w.is_leaf

True

此处我们介绍另一种方法，`.data`来返回可微张量的取值，从在避免在修改的过程中被追踪

In [30]:
w = torch.tensor(2., requires_grad = True)
w

tensor(2., requires_grad=True)

In [31]:
w.data # 查看张量的数值

tensor(2.)

In [32]:
w              # 但不改变张量本身可微性

tensor(2., requires_grad=True)

In [33]:
w.data -= w * 2  # 对其数值进行修改


In [34]:
w

tensor(-2., requires_grad=True)

In [35]:
w.is_leaf       # 张量仍然是叶节点

True

---

- Stage.4 训练模型

In [36]:
# 设置随机数种子
torch.manual_seed(420)    

# 初始化核心参数
batch_size = 10                                # 每一个小批的数量
lr = 0.03                                      # 学习率
num_epochs = 3                                 # 训练过程遍历几次数据
w = torch.zeros(3, 1, requires_grad = True)    # 随机设置初始权重

# 参与训练的模型方程
net = linreg                                   # 使用回归方程
loss = squared_loss                            # MSE作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size,features,labels):
        l = loss(net(X,w),y)
        l.backward()
        sgd(w,lr)
    train_l = loss(net(features,w),labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l))
    

epoch 1, loss 14.963247
epoch 2, loss 47.127636
epoch 3, loss 64.797066


In [37]:
net(X,w)  # 返回预测结果

tensor([[ 8.4490],
        [12.5513],
        [10.9238],
        [ 8.7683],
        [21.7664],
        [16.9534],
        [ 8.1466],
        [ 2.2295],
        [ 7.1768],
        [ 7.9883]], grad_fn=<MmBackward0>)

In [38]:
w

tensor([[ 3.0100],
        [-5.9828],
        [ 7.4010]], requires_grad=True)

当然，我们也可以使用tensorboard记录上述迭代过程中loss的变化过程

In [39]:
#————————————————————————————tensorboard记录loss的变化————————————————————————————
writer = SummaryWriter(log_dir='reg_loss')

In [40]:
#————————————————————————————tensorboard记录loss的变化————————————————————————————


# 初始化核心参数
batch_size = 10                                # 每一个小批的数量
lr = 0.03                                      # 学习率
num_epochs = 3                                 # 训练过程遍历几次数据
w = torch.zeros(3, 1, requires_grad = True)    # 随机设置初始权重

# 参与训练的模型方程
net = linreg                                   # 使用回归方程
loss = squared_loss                            # 均方误差的一半作为损失函数

# 模型训练过程
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w), y)
        l.backward()
        sgd(w, lr)
    train_l = loss(net(features, w), labels)
    # 记录训练误差
    writer.add_scalar('mul1', train_l, epoch) 

## 三、线性回归的快速实现

- 调库实现 --> 定义类Class LR
1. 定义模型 `Class LR(nn.Module):`
   1. `def __init__()`
   2. `def forward()`
   3. 实例化类
2. 定义损失函数`nn.MSELoss()`
3. 定义优化方法`optim.SGD(模型.paramerters(), lr = 学习率)`
4. 模型训练
   1. ```
      def fit(net, criterion, optimizer, batchdata, epochs):
        for epoch in range(epochs):
            for X, y in batchdata:
                yhat = net.forward(X)
                loss = criterion(yhat, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step() 
      ```
   2. `fit()` 训练模型
- `list(模型.parameters())`
        

- 定义核心参数
  

In [41]:
batch_size = 10                                # 每一个小批的数量
lr = 0.03                                      # 学习率
num_epochs = 3   

- 数据准备

In [42]:
# 设置随机数种子
torch.manual_seed(420)   

# 创建数据集
features, labels = tensorGenReg()
features = features[:, :-1]                                  # 剔除最后全是1的列
data = TensorDataset(features, labels)                       # 数据封装
batchData = DataLoader(data, batch_size = batch_size, shuffle = True)      # 数据加载

In [43]:
features

tensor([[-0.0070,  0.5044],
        [ 0.6704, -0.3829],
        [ 0.0302,  0.3826],
        ...,
        [-0.9164, -0.6087],
        [ 0.7815,  1.2865],
        [ 1.4819,  1.1390]])

- Stage 1.定义模型

In [44]:
class LR(nn.Module):
    def __init__(self, in_features=2, out_features=1):       # 定义模型的点线结构
        super(LR, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        
    def forward(self, x):                                    # 定义模型的正向传播规则
        out = self.linear(x)             
        return out

# 实例化模型
LR_model = LR()


- Stage 2.定义损失函数

In [45]:
criterion = nn.MSELoss()

- Stage 3.定义优化方法

In [46]:
optimizer = optim.SGD(LR_model.parameters(), lr = 0.03)

- Stage 4.模型训练

In [47]:
def fit(net, criterion, optimizer, batchdata, epochs):
    for epoch in range(epochs):
        for X, y in batchdata:
            yhat = net.forward(X)
            loss = criterion(yhat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        writer.add_scalar('loss', loss, global_step=epoch)          

> 当然，由于上述模型只有一层，因此也可以通过nn.Linear(2, 1)函数直接建模。

In [48]:
#————————————————————————————训练模型————————————————————————————
torch.manual_seed(420)   

fit(net = LR_model, 
    criterion = criterion, 
    optimizer = optimizer, 
    batchdata = batchData, 
    epochs = num_epochs)

In [49]:
LR_model

LR(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)

In [50]:
# 查看模型参数
list(LR_model.parameters())

[Parameter containing:
 tensor([[ 2.0006, -1.0000]], requires_grad=True),
 Parameter containing:
 tensor([1.0008], requires_grad=True)]

In [51]:
# 计算MSE
criterion(LR_model(features), labels)

tensor(0.0001, grad_fn=<MseLossBackward0>)

由于数据本身就是按照$y=2x_1-x_2+1$基本规律加上扰动项构建的，因此通过训练完成的参数可以看出模型效果较好。

当然，我们也可以通过add_graph方法，在writer中添加上述模型的记录图

- `writer.add_graph(模型, (训练模型的数据,))`

In [52]:
writer.add_graph(LR_model, (features,))

--- 


#### 简单线性回归局限性

此处我们进一步进行简单实验，当自变量和因变量满足最高次方为2次方的多项式函数关系时，或者扰动项增加时，简单线性回归误差将迅速增大。


In [53]:
#————————————————————————————自变量和因变量的最高次为2次的情况————————————————————————————
# Y = 2x1^2 - x2^2+1的情况
# 设置随机数种子
torch.manual_seed(420)   

# 创建数据集
features, labels = tensorGenReg(deg=2)
features = features[:, :-1]                                  # 剔除最后全是1的列
data = TensorDataset(features, labels)
batchData = DataLoader(data, batch_size = batch_size, shuffle = True)

# 模型实例化
LR_model = LR()

# 定义优化算法
optimizer = optim.SGD(LR_model.parameters(), lr = 0.03)

# 模型训练
fit(net = LR_model, 
    criterion = criterion, 
    optimizer = optimizer, 
    batchdata = batchData, 
    epochs = num_epochs)

# MSE结果查看
criterion(LR_model(features), labels)

tensor(10.1917, grad_fn=<MseLossBackward0>)

In [55]:
#————————————————————————————扰动项delta增加的情况————————————————————————————
# 设置随机数种子
torch.manual_seed(420)   

# 创建数据集
features, labels = tensorGenReg(delta=2)
features = features[:, :-1]                                  # 剔除最后全是1的列
data = TensorDataset(features, labels)
batchData = DataLoader(data, batch_size = batch_size, shuffle = True)

# 模型实例化
LR_model = LR()

# 定义优化算法
optimizer = optim.SGD(LR_model.parameters(), lr = 0.03)

# 模型训练
fit(net = LR_model, 
    criterion = criterion, 
    optimizer = optimizer, 
    batchdata = batchData, 
    epochs = num_epochs)

# MSE结果查看
criterion(LR_model(features), labels)

tensor(4.0471, grad_fn=<MseLossBackward0>)